In [36]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [37]:
import pickle
import pandas as pd
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
path='/content/gdrive/MyDrive/Top_C.csv'
df_test=pd.read_csv(path, header=None)

In [39]:
df_test

,0
0,\nBzzzzz! Looks like you forgot to say where y...
1,A group of honeybees looking for a new home. S...
2,This is how honeybee colonies (called “super o...
3,They are just having a family meeting about th...
4,Call the local bee keeper. They'd luv to get t...
...,...
72,I bee-lieve this is called festooning (forgive...
73,Queen stopped for a rest she will move on soon...
74,"Call a bee keeper, they know what to do to rem..."
75,They’re swarming just looking for a new home


In [40]:
df_test.shape

(77, 1)

In [41]:
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
import re
import string

In [42]:
# Load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer()
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

In [43]:
df_test_clean = df_test
df_test_clean[0] = df_test[0].apply(clean_string)

In [44]:
df_test_clean.head()

,0
0,bzzzzz looks like forgot say found bug theres ...
1,group honeybees looking new home somewhere que...
2,honeybee colonies called “super organisms” rep...
3,family meeting cheapest place live
4,call local bee keeper theyd luv get hands


In [46]:
import joblib
cv = joblib.load("/content/gdrive/MyDrive/vectorizer.pkl")
df_test_clean[0]=cv.transform(df_test_clean[0]).toarray()

In [47]:
model_embed='/content/gdrive/MyDrive/model2.pkl'
# file_name='model.pkl'
# path_to_embed=os.path.join(file_path, file_name)
model=pickle.load(open(model_embed,'rb'))

In [50]:
model

RandomForestClassifier()

In [51]:
prediction=model.predict(df_test_clean)

In [52]:
prediction

array([ 1.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,
        1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1., -1.,
        0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,
        0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
        0.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.])